In [85]:
from pyspark.sql import SparkSession

In [86]:
spark = SparkSession.builder.appName('rheeza').getOrCreate()

## Understand the dataset

In [87]:
trials_df = spark.read.json('dataset.json', multiLine = True) ## extracting the dataset

In [88]:
trials_df.printSchema()  ## examining the structure

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [89]:
trials_df.columns  ## examining the columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [90]:
## flattening the column

columns = ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [91]:
## flattening the columns
trials_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [92]:
from pyspark.sql import functions as fn

In [93]:
##counting null values per column using list comprehension method.

trials_df.select([ fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



## CLEANING

- Flatten the dataframe
- Rename columns
- Address null columns


In [94]:
new_trials_df = trials_df.select(columns)

In [75]:
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [95]:
# rename columns

new_column_names = {'ageofparticipant':'age_of_participant', 'name':'name_of_clinician', 'branch': 'clinic_branch', 'role':'role_of_clinician', 'experimentenddate': 'experiment_end_date', 'experimentstartdate':'experiment_start_date', 'noofhourspassedatfirstreaction': 'no_of_hours_passed_at_first_reaction', 'sideeffectsonparticipant' :'observed_side_effects'}

new_trials_df = new_trials_df.withColumnsRenamed(new_column_names)
new_trials_df.show(2)


+------------------+-------------+-----------------+-----------------+---------+-------------------+---------------------+------------------------------------+-------------+---------------------+
|age_of_participant|clinic_branch|name_of_clinician|role_of_clinician|drug_used|experiment_end_date|experiment_start_date|no_of_hours_passed_at_first_reaction|   conclusion|observed_side_effects|
+------------------+-------------+-----------------+-----------------+---------+-------------------+---------------------+------------------------------------+-------------+---------------------+
|                19|      Ontario|             Saul|        therapist|  Placebo|      1619827200000|        1617235200000|                                  52|BP normalized|       rashes on neck|
|                14|      Ontario|             Saul|            nurse| Naproxen|      1619827200000|        1617235200000|                                  78|    Follow-up|                  N/A|
+------------------+

In [100]:
# fill columns with null values

new_trials_final = new_trials_df.na.fill({'conclusion': 'unknown', 'role_of_clinician': 'unknown', 'no_of_hours_passed_at_first_reaction': 0})

In [102]:
new_trials_final.describe().show()

+-------+------------------+-------------+-----------------+-----------------+---------+--------------------+---------------------+------------------------------------+-------------+---------------------+
|summary|age_of_participant|clinic_branch|name_of_clinician|role_of_clinician|drug_used| experiment_end_date|experiment_start_date|no_of_hours_passed_at_first_reaction|   conclusion|observed_side_effects|
+-------+------------------+-------------+-----------------+-----------------+---------+--------------------+---------------------+------------------------------------+-------------+---------------------+
|  count|              3586|         3586|             3586|             3586|     3586|                3586|                 3586|                                3586|         3586|                 3586|
|   mean|17.507250418293363|         null|             null|             null|     null|1.618381578137200...| 1.615813671834913...|                    43.9771332961517|         nul